In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu

In [ ]:
!pip install -q langchain langchain-community

In [ ]:
from getpass import getpass
ACCESS_TOKEN = getpass("GITHUB_PERSONAL_TOKEN")

In [ ]:
from langchain.document_loaders import GitHubIssuesLoader

loader = GitHubIssuesLoader(
    repo="huggingface/peft",
    access_token=ACCESS_TOKEN,
    include_prs=False,
    state="all"
)

docs = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)

chunked_docs = splitter.split_documents(docs)

In [ ]:
!pip install faiss-cpu

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5'))

In [ ]:
pip install faiss-cpu

In [ ]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [ ]:
import importlib.metadata as m, transformers, os, sys, torch
print("transformers:", m.version("transformers"))
print("file:", transformers.__file__)
print("tokenizers:", m.version("tokenizers"))
print("accelerate:", m.version("accelerate"))
print("bitsandbytes:", m.version("bitsandbytes") if "bitsandbytes" in sys.modules else "not imported")
print("CUDA available:", torch.cuda.is_available())


In [ ]:
!pip uninstall -y transformers tokenizers accelerate
!pip install --no-cache-dir "transformers==4.45.2" "tokenizers>=0.19.1" "accelerate>=0.33.0" safetensors einops sentencepiece
# if you want 4-bit
!pip install --no-cache-dir bitsandbytes

In [ ]:
!pip uninstall -y transformers tokenizers accelerate
!pip install --no-cache-dir "transformers==4.45.2" "tokenizers>=0.19.1" "accelerate>=0.33.0" safetensors einops sentencepiece bitsandbytes-windows

In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install -U bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb,
    device_map="auto",
)

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)